# Test conversion on Niel's alpha-cyldextrin bound to 1-butylamine[+1] file!
```
/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/a-bam-p/a00
```

In [566]:
%load_ext autoreload
%autoreload 2

import numpy as np
import subprocess as sp

from openeye.oechem import *
from openforcefield.typing.engines.smirnoff import *
from openforcefield.utils import mergeStructure
import parmed as pmd

from utils import create_pdb_with_conect, prune_conect
from utils import split_topology, create_host_guest_topology
from utils import create_host_mol2
from utils import load_mol2, check_unique_atom_names
from utils import extract_water_and_ions, create_water_and_ions_parameters
from utils import extract_dummy_atoms, create_dummy_atom_parameters
from utils import residue_mapping

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
create_pdb_with_conect(solvated_pdb='original/a-bam-p/full.crds', 
                       amber_prmtop='original/a-bam-p/full.topo', 
                       output_pdb='generated/a-bam-p/full.pdb')
prune_conect(input_pdb='full.pdb', 
             output_pdb='full_conect.pdb', 
             path='generated/a-bam-p/')

PDB file written by cpptraj.
First water residue = 160
Found first water CONECT entry at line = 8322


In [20]:
components = split_topology(file_name='generated/a-bam-p/full.pdb')
hg_topology = create_host_guest_topology(components, host_resname='MGO', guest_resname='BAM')

In [21]:
host = load_mol2(filename='original/a-bam-p/MGO.mol2', 
                 name='MGO', 
                 add_tripos=True)
guest = load_mol2(filename='original/a-bam-p/bam.mol2', 
                  name='BAM', 
                  add_tripos=False)
check_unique_atom_names(host)
check_unique_atom_names(guest)
molecules = [host, guest]

21 atoms in structure, 21 unique atom names.
17 atoms in structure, 17 unique atom names.


In [22]:
create_host_mol2('generated/a-bam-p/full.pdb', 'original/a-bam-p/full.topo', 'MGO', 'generated/a-bam-p/MGO.mol2')

MOL2 file written by cpptraj.


In [23]:
host = load_mol2(filename='generated/a-bam-p/MGO.mol2', 
                 name='MGO', 
                 add_tripos=True)
guest = load_mol2(filename='original/a-bam-p/bam.mol2', 
                  name='BAM', 
                  add_tripos=False)
check_unique_atom_names(host)
check_unique_atom_names(guest)
molecules = [host, guest]

126 atoms in structure, 126 unique atom names.
17 atoms in structure, 17 unique atom names.


In [24]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml') 
system = ff.createSystem(hg_topology.topology, molecules,
                         nonbondedCutoff=1.1*unit.nanometer, 
                         ewaldErrorTolerance=1e-4
                         )

In [25]:
hg_structure = pmd.openmm.topsystem.load_topology(hg_topology.topology, system, hg_topology.positions)

In [26]:
hg_structure.save('generated/a-bam-p/hg.prmtop')

OSError: generated/a-bam-p/hg.prmtop exists; not overwriting

In [27]:
hg_structure.save('generated/a-bam-p/hg.inpcrd')

OSError: generated/a-bam-p/hg.inpcrd exists; not overwriting

In [28]:
# extract_dummy_atoms(amber_prmtop='original/a-bam-p/full.topo',
#                       amber_inpcrd='original/a-bam-p/full.crds',
#                       dummy_residue=':DUM',
#                      output_pdb='generated/a-bam-p/dummy.pdb')

In [29]:
# create_dummy_atom_parameters(input_pdb='dummy.pdb',
#                                 output_prmtop='dummy.prmtop',
#                                 output_inpcrd='dummy.inpcrd',
#                                 path='generated/a-bam-p/')

In [30]:
# extract_water_and_ions(amber_prmtop='original/a-bam-p/full.topo',
#                       amber_inpcrd='original/a-bam-p/full.crds',
#                       host_residue=':MGO',
#                       guest_residue=':BAM',
#                       dummy=':DUM',
#                       output_pdb='generated/a-bam-p/water_ions.pdb')

extract_water_and_ions(amber_prmtop='original/a-bam-p/full.topo',
                      amber_inpcrd='original/a-bam-p/full.crds',
                      host_residue=':MGO',
                      guest_residue=':BAM',
                      dummy=None,
                      output_pdb='generated/a-bam-p/water_ions.pdb')

Water and ion PDB file written by cpptraj.


In [31]:
# create_water_and_ions_parameters(input_pdb='water_ions.pdb',
#                                 output_prmtop='water_ions.prmtop',
#                                 output_inpcrd='water_ions.inpcrd',
#                                 dummy_atoms=False,
#                                 path='generated/a-bam-p/')

create_water_and_ions_parameters(input_pdb='water_ions.pdb',
                                output_prmtop='water_ions.prmtop',
                                output_inpcrd='water_ions.inpcrd',
                                dummy_atoms=True,
                                path='generated/a-bam-p/')

Writing dummy atom `frcmod`.
Writing dummy atom `mol2`.
Water and ion parameters and coordinates written by tleap.


In [32]:
# dummy = pmd.amber.AmberParm('generated/a-bam-p/dummy.prmtop', xyz='generated/a-bam-p/dummy.inpcrd')

In [33]:
water_and_ions = pmd.amber.AmberParm('generated/a-bam-p/water_ions.prmtop', xyz='generated/a-bam-p/water_ions.inpcrd')

In [34]:
# merged = mergeStructure(dummy, hg_structure)

In [35]:
# merged.save('generated/a-bam-p/dum-hg.prmtop')

In [36]:
# merged.save('generated/a-bam-p/dum-hg.inpcrd')

In [38]:
# merged_2 = mergeStructure(merged, water_and_ions)
merged_2 = mergeStructure(hg_structure, water_and_ions)

In [40]:
merged_2.save('generated/a-bam-p/solvated_smirnoff.prmtop')

In [41]:
merged_2.save('generated/a-bam-p/solvated_smirnoff.inpcrd')

This is still a problem because the atom index has changed and that's because the dummy atoms in my file are *after* the host and guest, but the dummy atoms are *before* the host and guest in the original coordinates. I think the easist way to deal with this is to extra them first, then merge.

In [132]:
merged_2.box

In [133]:
merged_2.coordinates

array([[ 19.42200089,  19.38299942,  26.52599907],
       [ 19.42200089,  19.38299942,  21.52599907],
       [ 19.42200089,  22.88299942,  18.02599907],
       ..., 
       [ 16.58300018,  18.46800041,  17.95100021],
       [ 17.37599945,  17.97299957,  17.74300003],
       [ 16.87700081,  19.14699936,  18.55800056]], dtype=float32)

In [134]:
merged_2.visualize

<bound method Structure.visualize of <AmberParm 6159 atoms; 2023 residues; 6148 bonds; parametrized>>

In [136]:
merged = dummy + hg_structure + water_and_ions

In [137]:
merged

<AmberParm 6159 atoms; 2023 residues; 6148 bonds; parametrized>

In [138]:
merged.save('generated/a-bam-p/tmp.prmtop')

In [139]:
merged.save('generated/a-bam-p/tmp.inpcrd')

In [141]:
prmtop = pmd.amber.AmberParm('generated/a-bam-p/tmp.prmtop')
pmd.tools.checkValidity(prmtop)

IndexError: list index out of range

The problem comes from separating out the dummy atoms. Let's *keep* the dummy atoms in place, and re-write the `disang` file with a mapping between the index numbers in my host and guest molecule and Niel's.

In [52]:
def residue_mapping

original = pmd.load_file('original/a-bam-p/full.topo', 'original/a-bam-p/full.crds')
new = pmd.load_file('generated/a-bam-p/solvated_smirnoff.prmtop', 'generated/a-bam-p/solvated_smirnoff.inpcrd')
from utils import atom_mapping
original_host = pmd.amber.mask.AmberMask(original, ':MGO')
new_host = pmd.amber.mask.AmberMask(new, ':MGO')
# atom_mapping(original_host, new_host)

In [53]:
from oeommtools.utils import *

In [54]:
openmmTop_to_oemol(original[':MGO'], original[':MGO'].positions)

AttributeError: 'AmberParm' object has no attribute 'chains'

In [55]:
openmmTop_to_oemol(original, original.positions)

AttributeError: 'AmberParm' object has no attribute 'chains'

In [59]:
original[':MGO'].save('tmp.mol2')

In [60]:
from utils import load_mol2

In [64]:
original_OEMol = load_mol2('tmp.mol2')

In [65]:
new[':MGO'].save('tmp2.mol2')

OSError: tmp2.mol2 exists; not overwriting

In [66]:
new_OEMol = load_mol2('tmp2.mol2')

In [84]:
mapping = atom_mapping(original_OEMol, new_OEMol)

Determining mapping...
Reference → Target
(C1   Du   )   0 →   0 (C1  )
(H1   Du   )   1 →   1 (H1  )
(Os1  Du   )   2 →   2 (O1  )
(C2   Du   )   3 →   3 (C2  )
(H2   Du   )   4 →   4 (H2  )
(O1   Du   )   5 →   5 (O2  )
(Ho1  Du   )   6 →   6 (H3  )
(C3   Du   )   7 →   7 (C3  )
(H3   Du   )   8 →   8 (H4  )
(O2   Du   )   9 →   9 (O3  )
(Ho2  Du   )  10 →  10 (H5  )
(C4   Du   )  11 →  11 (C4  )
(H4   Du   )  12 →  12 (H6  )
(C5   Du   )  13 →  13 (C5  )
(H5   Du   )  14 →  14 (H7  )
(Os2  Du   )  15 →  15 (O4  )
(C6   Du   )  16 →  16 (C6  )
(H6   Du   )  17 →  17 (H8  )
(H7   Du   )  18 →  18 (H9  )
(O3   Du   )  19 →  19 (O5  )
(Ho3  Du   )  20 →  20 (H10 )
(C10  Du   )  32 →  32 (C10 )
(C9   Du   )  28 →  28 (C9  )
(C8   Du   )  24 →  24 (C8  )
(C7   Du   )  21 →  21 (C7  )
(H8   Du   )  22 →  22 (H11 )
(Os3  Du   )  23 →  23 (O6  )
(H9   Du   )  25 →  25 (H12 )
(O4   Du   )  26 →  26 (O7  )
(Ho4  Du   )  27 →  27 (H13 )
(H10  Du   )  29 →  29 (H14 )
(O5   Du   )  30 →  30 (O8  

In [69]:
host_map = residue_mapping(input_inpcrd='original/a-bam-p/full.crds',
                          input_topology='original/a-bam-p/full.topo',
                          target_inpcrd='generated/a-bam-p/solvated_smirnoff.inpcrd',
                          target_topology='generated/a-bam-p/solvated_smirnoff.prmtop',
                          mask=':MGO')

Determining mapping...
Reference → Target
(C1   Du   )   0 →   0 (C1  )
(H1   Du   )   1 →   1 (H1  )
(Os1  Du   )   2 →   2 (O1  )
(C2   Du   )   3 →   3 (C2  )
(H2   Du   )   4 →   4 (H2  )
(O1   Du   )   5 →   5 (O2  )
(Ho1  Du   )   6 →   6 (H3  )
(C3   Du   )   7 →   7 (C3  )
(H3   Du   )   8 →   8 (H4  )
(O2   Du   )   9 →   9 (O3  )
(Ho2  Du   )  10 →  10 (H5  )
(C4   Du   )  11 →  11 (C4  )
(H4   Du   )  12 →  12 (H6  )
(C5   Du   )  13 →  13 (C5  )
(H5   Du   )  14 →  14 (H7  )
(Os2  Du   )  15 →  15 (O4  )
(C6   Du   )  16 →  16 (C6  )
(H6   Du   )  17 →  17 (H8  )
(H7   Du   )  18 →  18 (H9  )
(O3   Du   )  19 →  19 (O5  )
(Ho3  Du   )  20 →  20 (H10 )
(C10  Du   )  32 →  32 (C10 )
(C9   Du   )  28 →  28 (C9  )
(C8   Du   )  24 →  24 (C8  )
(C7   Du   )  21 →  21 (C7  )
(H8   Du   )  22 →  22 (H11 )
(Os3  Du   )  23 →  23 (O6  )
(H9   Du   )  25 →  25 (H12 )
(O4   Du   )  26 →  26 (O7  )
(Ho4  Du   )  27 →  27 (H13 )
(H10  Du   )  29 →  29 (H14 )
(O5   Du   )  30 →  30 (O8  

In [71]:
guest_map = residue_mapping(input_inpcrd='original/a-bam-p/full.crds',
                          input_topology='original/a-bam-p/full.topo',
                          target_inpcrd='generated/a-bam-p/solvated_smirnoff.inpcrd',
                          target_topology='generated/a-bam-p/solvated_smirnoff.prmtop',
                          mask=':BAM')

Determining mapping...
Reference → Target
(N1   Du   )   0 →   0 (N1  )
(H1   Du   )   1 →   1 (H1  )
(H2   Du   )   2 →   2 (H2  )
(H3   Du   )   3 →   3 (H3  )
(C1   Du   )   4 →   4 (C1  )
(H4   Du   )   5 →   5 (H4  )
(H5   Du   )   6 →   6 (H5  )
(C2   Du   )   7 →   7 (C2  )
(H6   Du   )   8 →   8 (H6  )
(H7   Du   )   9 →   9 (H7  )
(C3   Du   )  10 →  10 (C3  )
(H8   Du   )  11 →  11 (H8  )
(H9   Du   )  12 →  12 (H9  )
(C4   Du   )  13 →  13 (C4  )
(H10  Du   )  14 →  14 (H10 )
(H11  Du   )  15 →  15 (H11 )
(H12  Du   )  16 →  16 (H12 )


In [82]:
dummy_map = residue_mapping(input_inpcrd='original/a-bam-p/full.crds',
                          input_topology='original/a-bam-p/full.topo',
                          target_inpcrd='generated/a-bam-p/solvated_smirnoff.inpcrd',
                          target_topology='generated/a-bam-p/solvated_smirnoff.prmtop',
                          mask=':DUM')

Determining mapping...
Mapping mask :DUM...
Found UNL...


Option 1 is to simply do the overall mapping.

Option 2 may be to go through PDB and, well, grep for atom names.

Option 3 may be to use PDB as an intermediate to the mapping function.

## Explore option 1, since we already have the overall mapping...

In [100]:
mapping = residue_mapping(input_inpcrd='original/a-bam-p/full.crds',
                          input_topology='original/a-bam-p/full.topo',
                          target_inpcrd='generated/a-bam-p/solvated_smirnoff.inpcrd',
                          target_topology='generated/a-bam-p/solvated_smirnoff.prmtop',
                          mask=None)
print(mapping)

Determining mapping...
Reference → Target
(C1   Du   )   3 →   0 (C1  )
(H1   Du   )   4 →   1 (H1  )
(Os1  Du   )   5 →   2 (O1  )
(C2   Du   )   6 →   3 (C2  )
(H2   Du   )   7 →   4 (H2  )
(O1   Du   )   8 →   5 (O2  )
(Ho1  Du   )   9 →   6 (H3  )
(C3   Du   )  10 →   7 (C3  )
(H3   Du   )  11 →   8 (H4  )
(O2   Du   )  12 →   9 (O3  )
(Ho2  Du   )  13 →  10 (H5  )
(C4   Du   )  14 →  11 (C4  )
(H4   Du   )  15 →  12 (H6  )
(C5   Du   )  16 →  13 (C5  )
(H5   Du   )  17 →  14 (H7  )
(Os2  Du   )  18 →  15 (O4  )
(C6   Du   )  19 →  16 (C6  )
(H6   Du   )  20 →  17 (H8  )
(H7   Du   )  21 →  18 (H9  )
(O3   Du   )  22 →  19 (O5  )
(Ho3  Du   )  23 →  20 (H10 )
(C10  Du   )  35 →  32 (C10 )
(C9   Du   )  31 →  28 (C9  )
(C8   Du   )  27 →  24 (C8  )
(C7   Du   )  24 →  21 (C7  )
(H8   Du   )  25 →  22 (H11 )
(Os3  Du   )  26 →  23 (O6  )
(H9   Du   )  28 →  25 (H12 )
(O4   Du   )  29 →  26 (O7  )
(Ho4  Du   )  30 →  27 (H13 )
(H10  Du   )  32 →  29 (H14 )
(O5   Du   )  33 →  30 (O8  

(H791 Du   ) 1264 → 1264 (H809)
(H792 Du   ) 1265 → 1265 (H810)
(O388 Du   ) 1266 → 1266 (O400)
(H793 Du   ) 1267 → 1267 (H811)
(H794 Du   ) 1268 → 1268 (H812)
(O389 Du   ) 1269 → 1269 (O401)
(H795 Du   ) 1270 → 1270 (H813)
(H796 Du   ) 1271 → 1271 (H814)
(O390 Du   ) 1272 → 1272 (O402)
(H797 Du   ) 1273 → 1273 (H815)
(H798 Du   ) 1274 → 1274 (H816)
(O391 Du   ) 1275 → 1275 (O403)
(H799 Du   ) 1276 → 1276 (H817)
(H800 Du   ) 1277 → 1277 (H818)
(O392 Du   ) 1278 → 1278 (O404)
(H801 Du   ) 1279 → 1279 (H819)
(H802 Du   ) 1280 → 1280 (H820)
(O393 Du   ) 1281 → 1281 (O405)
(H803 Du   ) 1282 → 1282 (H821)
(H804 Du   ) 1283 → 1283 (H822)
(O394 Du   ) 1284 → 1284 (O406)
(H805 Du   ) 1285 → 1285 (H823)
(H806 Du   ) 1286 → 1286 (H824)
(O395 Du   ) 1287 → 1287 (O407)
(H807 Du   ) 1288 → 1288 (H825)
(H808 Du   ) 1289 → 1289 (H826)
(O396 Du   ) 1290 → 1290 (O408)
(H809 Du   ) 1291 → 1291 (H827)
(H810 Du   ) 1292 → 1292 (H828)
(O397 Du   ) 1293 → 1293 (O409)
(H811 Du   ) 1294 → 1294 (H829)
(H812 Du

(O742 Du   ) 2328 → 2328 (O754)
(H1501 Du   ) 2329 → 2329 (H1519)
(H1502 Du   ) 2330 → 2330 (H1520)
(O743 Du   ) 2331 → 2331 (O755)
(H1503 Du   ) 2332 → 2332 (H1521)
(H1504 Du   ) 2333 → 2333 (H1522)
(O744 Du   ) 2334 → 2334 (O756)
(H1505 Du   ) 2335 → 2335 (H1523)
(H1506 Du   ) 2336 → 2336 (H1524)
(O745 Du   ) 2337 → 2337 (O757)
(H1507 Du   ) 2338 → 2338 (H1525)
(H1508 Du   ) 2339 → 2339 (H1526)
(O746 Du   ) 2340 → 2340 (O758)
(H1509 Du   ) 2341 → 2341 (H1527)
(H1510 Du   ) 2342 → 2342 (H1528)
(O747 Du   ) 2343 → 2343 (O759)
(H1511 Du   ) 2344 → 2344 (H1529)
(H1512 Du   ) 2345 → 2345 (H1530)
(O748 Du   ) 2346 → 2346 (O760)
(H1513 Du   ) 2347 → 2347 (H1531)
(H1514 Du   ) 2348 → 2348 (H1532)
(O749 Du   ) 2349 → 2349 (O761)
(H1515 Du   ) 2350 → 2350 (H1533)
(H1516 Du   ) 2351 → 2351 (H1534)
(O750 Du   ) 2352 → 2352 (O762)
(H1517 Du   ) 2353 → 2353 (H1535)
(H1518 Du   ) 2354 → 2354 (H1536)
(O751 Du   ) 2355 → 2355 (O763)
(H1519 Du   ) 2356 → 2356 (H1537)
(H1520 Du   ) 2357 → 2357 (H1538)


(H2206 Du   ) 3386 → 3386 (H2224)
(O1095 Du   ) 3387 → 3387 (O1107)
(H2207 Du   ) 3388 → 3388 (H2225)
(H2208 Du   ) 3389 → 3389 (H2226)
(O1096 Du   ) 3390 → 3390 (O1108)
(H2209 Du   ) 3391 → 3391 (H2227)
(H2210 Du   ) 3392 → 3392 (H2228)
(O1097 Du   ) 3393 → 3393 (O1109)
(H2211 Du   ) 3394 → 3394 (H2229)
(H2212 Du   ) 3395 → 3395 (H2230)
(O1098 Du   ) 3396 → 3396 (O1110)
(H2213 Du   ) 3397 → 3397 (H2231)
(H2214 Du   ) 3398 → 3398 (H2232)
(O1099 Du   ) 3399 → 3399 (O1111)
(H2215 Du   ) 3400 → 3400 (H2233)
(H2216 Du   ) 3401 → 3401 (H2234)
(O1100 Du   ) 3402 → 3402 (O1112)
(H2217 Du   ) 3403 → 3403 (H2235)
(H2218 Du   ) 3404 → 3404 (H2236)
(O1101 Du   ) 3405 → 3405 (O1113)
(H2219 Du   ) 3406 → 3406 (H2237)
(H2220 Du   ) 3407 → 3407 (H2238)
(O1102 Du   ) 3408 → 3408 (O1114)
(H2221 Du   ) 3409 → 3409 (H2239)
(H2222 Du   ) 3410 → 3410 (H2240)
(O1103 Du   ) 3411 → 3411 (O1115)
(H2223 Du   ) 3412 → 3412 (H2241)
(H2224 Du   ) 3413 → 3413 (H2242)
(O1104 Du   ) 3414 → 3414 (O1116)
(H2225 Du   ) 

(H2856 Du   ) 4361 → 4361 (H2874)
(O1420 Du   ) 4362 → 4362 (O1432)
(H2857 Du   ) 4363 → 4363 (H2875)
(H2858 Du   ) 4364 → 4364 (H2876)
(O1421 Du   ) 4365 → 4365 (O1433)
(H2859 Du   ) 4366 → 4366 (H2877)
(H2860 Du   ) 4367 → 4367 (H2878)
(O1422 Du   ) 4368 → 4368 (O1434)
(H2861 Du   ) 4369 → 4369 (H2879)
(H2862 Du   ) 4370 → 4370 (H2880)
(O1423 Du   ) 4371 → 4371 (O1435)
(H2863 Du   ) 4372 → 4372 (H2881)
(H2864 Du   ) 4373 → 4373 (H2882)
(O1424 Du   ) 4374 → 4374 (O1436)
(H2865 Du   ) 4375 → 4375 (H2883)
(H2866 Du   ) 4376 → 4376 (H2884)
(O1425 Du   ) 4377 → 4377 (O1437)
(H2867 Du   ) 4378 → 4378 (H2885)
(H2868 Du   ) 4379 → 4379 (H2886)
(O1426 Du   ) 4380 → 4380 (O1438)
(H2869 Du   ) 4381 → 4381 (H2887)
(H2870 Du   ) 4382 → 4382 (H2888)
(O1427 Du   ) 4383 → 4383 (O1439)
(H2871 Du   ) 4384 → 4384 (H2889)
(H2872 Du   ) 4385 → 4385 (H2890)
(O1428 Du   ) 4386 → 4386 (O1440)
(H2873 Du   ) 4387 → 4387 (H2891)
(H2874 Du   ) 4388 → 4388 (H2892)
(O1429 Du   ) 4389 → 4389 (O1441)
(H2875 Du   ) 

(H3363 Du   ) 5122 → 5122 (H3381)
(H3364 Du   ) 5123 → 5123 (H3382)
(O1674 Du   ) 5124 → 5124 (O1686)
(H3365 Du   ) 5125 → 5125 (H3383)
(H3366 Du   ) 5126 → 5126 (H3384)
(O1675 Du   ) 5127 → 5127 (O1687)
(H3367 Du   ) 5128 → 5128 (H3385)
(H3368 Du   ) 5129 → 5129 (H3386)
(O1676 Du   ) 5130 → 5130 (O1688)
(H3369 Du   ) 5131 → 5131 (H3387)
(H3370 Du   ) 5132 → 5132 (H3388)
(O1677 Du   ) 5133 → 5133 (O1689)
(H3371 Du   ) 5134 → 5134 (H3389)
(H3372 Du   ) 5135 → 5135 (H3390)
(O1678 Du   ) 5136 → 5136 (O1690)
(H3373 Du   ) 5137 → 5137 (H3391)
(H3374 Du   ) 5138 → 5138 (H3392)
(O1679 Du   ) 5139 → 5139 (O1691)
(H3375 Du   ) 5140 → 5140 (H3393)
(H3376 Du   ) 5141 → 5141 (H3394)
(O1680 Du   ) 5142 → 5142 (O1692)
(H3377 Du   ) 5143 → 5143 (H3395)
(H3378 Du   ) 5144 → 5144 (H3396)
(O1681 Du   ) 5145 → 5145 (O1693)
(H3379 Du   ) 5146 → 5146 (H3397)
(H3380 Du   ) 5147 → 5147 (H3398)
(O1682 Du   ) 5148 → 5148 (O1694)
(H3381 Du   ) 5149 → 5149 (H3399)
(H3382 Du   ) 5150 → 5150 (H3400)
(O1683 Du   ) 

(H4002 Du   ) 6080 → 6080 (H4020)
(O1993 Du   ) 6081 → 6081 (O2005)
(H4003 Du   ) 6082 → 6082 (H4021)
(H4004 Du   ) 6083 → 6083 (H4022)
(O1994 Du   ) 6084 → 6084 (O2006)
(H4005 Du   ) 6085 → 6085 (H4023)
(H4006 Du   ) 6086 → 6086 (H4024)
(O1995 Du   ) 6087 → 6087 (O2007)
(H4007 Du   ) 6088 → 6088 (H4025)
(H4008 Du   ) 6089 → 6089 (H4026)
(O1996 Du   ) 6090 → 6090 (O2008)
(H4009 Du   ) 6091 → 6091 (H4027)
(H4010 Du   ) 6092 → 6092 (H4028)
(O1997 Du   ) 6093 → 6093 (O2009)
(H4011 Du   ) 6094 → 6094 (H4029)
(H4012 Du   ) 6095 → 6095 (H4030)
(O1998 Du   ) 6096 → 6096 (O2010)
(H4013 Du   ) 6097 → 6097 (H4031)
(H4014 Du   ) 6098 → 6098 (H4032)
(O1999 Du   ) 6099 → 6099 (O2011)
(H4015 Du   ) 6100 → 6100 (H4033)
(H4016 Du   ) 6101 → 6101 (H4034)
(O2000 Du   ) 6102 → 6102 (O2012)
(H4017 Du   ) 6103 → 6103 (H4035)
(H4018 Du   ) 6104 → 6104 (H4036)
(O2001 Du   ) 6105 → 6105 (O2013)
(H4019 Du   ) 6106 → 6106 (H4037)
(H4020 Du   ) 6107 → 6107 (H4038)
(O2002 Du   ) 6108 → 6108 (O2014)
(H4021 Du   ) 

In [101]:
mapping

{3: 0,
 4: 1,
 5: 2,
 6: 3,
 7: 4,
 8: 5,
 9: 6,
 10: 7,
 11: 8,
 12: 9,
 13: 10,
 14: 11,
 15: 12,
 16: 13,
 17: 14,
 18: 15,
 19: 16,
 20: 17,
 21: 18,
 22: 19,
 23: 20,
 35: 32,
 31: 28,
 27: 24,
 24: 21,
 25: 22,
 26: 23,
 28: 25,
 29: 26,
 30: 27,
 32: 29,
 33: 30,
 34: 31,
 36: 33,
 37: 34,
 38: 35,
 39: 36,
 40: 37,
 41: 38,
 42: 39,
 43: 40,
 44: 41,
 56: 53,
 52: 49,
 48: 45,
 45: 42,
 46: 43,
 47: 44,
 49: 46,
 50: 47,
 51: 48,
 53: 50,
 54: 51,
 55: 52,
 57: 54,
 58: 55,
 59: 56,
 60: 57,
 61: 58,
 62: 59,
 63: 60,
 64: 61,
 65: 62,
 77: 74,
 73: 70,
 69: 66,
 66: 63,
 67: 64,
 68: 65,
 70: 67,
 71: 68,
 72: 69,
 74: 71,
 75: 72,
 76: 73,
 78: 75,
 79: 76,
 80: 77,
 81: 78,
 82: 79,
 83: 80,
 84: 81,
 85: 82,
 86: 83,
 98: 95,
 94: 91,
 90: 87,
 87: 84,
 88: 85,
 89: 86,
 91: 88,
 92: 89,
 93: 90,
 95: 92,
 96: 93,
 97: 94,
 99: 96,
 100: 97,
 101: 98,
 102: 99,
 103: 100,
 104: 101,
 105: 102,
 106: 103,
 107: 104,
 110: 107,
 108: 105,
 109: 106,
 111: 108,
 112: 109,
 113

In [115]:
np.genfromtxt('original/a-bam-p/disang.rest', delimiter='=', skip_header=1, dtype=None)

array([[b'&rst iat', b' 1,13,            r1', b'     0.0000, r2',
        b'     7.4902, r3', b'     7.4902, r4', b'   999.0000, rk2',
        b'   5.0000000, rk3', b'   5.0000000, &end'],
       [b'&rst iat', b' 2,1,13,          r1', b'     0.0000, r2',
        b'   131.3655, r3', b'   131.3655, r4', b'   180.0000, rk2',
        b' 100.0000000, rk3', b' 100.0000000, &end'],
       [b'&rst iat', b' 3,2,1,13,        r1', b'   -64.8846, r2',
        b'   115.1154, r3', b'   115.1154, r4', b'   295.1154, rk2',
        b' 100.0000000, rk3', b' 100.0000000, &end'],
       [b'&rst iat', b' 1,13,46,         r1', b'     0.0000, r2',
        b'    55.1110, r3', b'    55.1110, r4', b'   180.0000, rk2',
        b' 100.0000000, rk3', b' 100.0000000, &end'],
       [b'&rst iat', b' 2,1,13,46,       r1', b'   -15.4090, r2',
        b'   164.5910, r3', b'   164.5910, r4', b'   344.5910, rk2',
        b' 100.0000000, rk3', b' 100.0000000, &end'],
       [b'&rst iat', b' 1,13,46,104,     r1', b'   -70.

In [125]:
with open('original/a-bam-p/disang.rest') as disang:
    lines = []
    for line in disang:
        lines.append(line)

In [160]:
for line in lines[1:]:
    # Restraint residues...
    old_restraint_list = line.split()[2]
    old_restraint_residues = [int(i) for i in old_restraint_list.split(',') if i is not '']

    # Make a list for the new residues...
    new_restraint_residues = []
    
    for atom_index in old_restraint_residues:
        new_restraint_residues.append(mapping[atom_index])
    
    assert len(old_restraint_residues) == len(new_restraint_residues)
    print(f'{old_restraint_list} → {list(new_restraint_residues)}')

1,13, → [144, 10]
2,1,13, → [145, 144, 10]
3,2,1,13, → [0, 145, 144, 10]
1,13,46, → [144, 10, 43]
2,1,13,46, → [145, 144, 10, 43]
1,13,46,104, → [144, 10, 43, 101]
1,143, → [144, 140]
2,1,143, → [145, 144, 140]
1,143,130, → [144, 140, 127]
19,4,6,36, → [16, 1, 3, 33]
40,25,27,57, → [37, 22, 24, 54]
61,46,48,78, → [58, 43, 45, 75]
82,67,69,99, → [79, 64, 66, 96]
103,88,90,120, → [100, 85, 87, 117]
124,109,111,15, → [121, 106, 108, 12]
4,6,36,38, → [1, 3, 33, 35]
25,27,57,59, → [22, 24, 54, 56]
46,48,78,80, → [43, 45, 75, 77]
67,69,99,101, → [64, 66, 96, 98]
88,90,120,122, → [85, 87, 117, 119]
109,111,15,17, → [106, 108, 12, 14]
9,143 → [6, 140]
23,143 → [20, 140]
30,143 → [27, 140]
44,143 → [41, 140]
51,143 → [48, 140]
65,143 → [62, 140]
72,143 → [69, 140]
86,143 → [83, 140]
93,143 → [90, 140]
107,143 → [104, 140]
114,143 → [111, 140]
128,143 → [125, 140]
2,143,130 → [145, 140, 127]


In [170]:
{k : v for k,v in mapping.items() if v in [0]}

{3: 0}

In [171]:
{k + 1 : v + 1for k,v in mapping.items() if v in [0]}

{4: 1}

In [178]:
amber_mapping = {k + 1 : v + 1 for k,v in mapping.items()}

In [182]:
amber_mapping[1]

144

In [184]:
mapping[0]

143

In [206]:
for line in lines[1:]:
    # Restraint residues...
    old_restraint_list = line.split()[2]
    old_restraint_residues = [int(i) for i in old_restraint_list.split(',') if i is not '']

    # Make a list for the new residues...
    new_restraint_residues = []
    
    for atom_index in old_restraint_residues:
        new_restraint_residues.append(mapping[atom_index - 1])
    
    assert len(old_restraint_residues) == len(new_restraint_residues)
    print(f'{old_restraint_list} → {new_restraint_residues}')

1,13, → [143, 9]
2,1,13, → [144, 143, 9]
3,2,1,13, → [145, 144, 143, 9]
1,13,46, → [143, 9, 42]
2,1,13,46, → [144, 143, 9, 42]
1,13,46,104, → [143, 9, 42, 100]
1,143, → [143, 139]
2,1,143, → [144, 143, 139]
1,143,130, → [143, 139, 126]
19,4,6,36, → [15, 0, 2, 32]
40,25,27,57, → [36, 21, 23, 53]
61,46,48,78, → [57, 42, 44, 74]
82,67,69,99, → [78, 63, 65, 95]
103,88,90,120, → [99, 84, 86, 116]
124,109,111,15, → [120, 105, 107, 11]
4,6,36,38, → [0, 2, 32, 34]
25,27,57,59, → [21, 23, 53, 55]
46,48,78,80, → [42, 44, 74, 76]
67,69,99,101, → [63, 65, 95, 97]
88,90,120,122, → [84, 86, 116, 118]
109,111,15,17, → [105, 107, 11, 13]
9,143 → [5, 139]
23,143 → [19, 139]
30,143 → [26, 139]
44,143 → [40, 139]
51,143 → [47, 139]
65,143 → [61, 139]
72,143 → [68, 139]
86,143 → [82, 139]
93,143 → [89, 139]
107,143 → [103, 139]
114,143 → [110, 139]
128,143 → [124, 139]
2,143,130 → [144, 139, 126]


In [222]:
for line in lines[1:]:
    # Restraint residues...
    old_restraint_list = line.split()[2]
    old_restraint_residues = [int(i) for i in old_restraint_list.split(',') if i is not '']

    # Make a list for the new residues...
    new_restraint_residues = []
    
    for atom_index in old_restraint_residues:
        new_restraint_residues.append(mapping[atom_index - 1])
    
    assert len(old_restraint_residues) == len(new_restraint_residues)
    print(f'{old_restraint_list} → ', end="")
    [print(f'{i},', end='') for i in new_restraint_residues]
    print('')

1,13, → 143,9,
2,1,13, → 144,143,9,
3,2,1,13, → 145,144,143,9,
1,13,46, → 143,9,42,
2,1,13,46, → 144,143,9,42,
1,13,46,104, → 143,9,42,100,
1,143, → 143,139,
2,1,143, → 144,143,139,
1,143,130, → 143,139,126,
19,4,6,36, → 15,0,2,32,
40,25,27,57, → 36,21,23,53,
61,46,48,78, → 57,42,44,74,
82,67,69,99, → 78,63,65,95,
103,88,90,120, → 99,84,86,116,
124,109,111,15, → 120,105,107,11,
4,6,36,38, → 0,2,32,34,
25,27,57,59, → 21,23,53,55,
46,48,78,80, → 42,44,74,76,
67,69,99,101, → 63,65,95,97,
88,90,120,122, → 84,86,116,118,
109,111,15,17, → 105,107,11,13,
9,143 → 5,139,
23,143 → 19,139,
30,143 → 26,139,
44,143 → 40,139,
51,143 → 47,139,
65,143 → 61,139,
72,143 → 68,139,
86,143 → 82,139,
93,143 → 89,139,
107,143 → 103,139,
114,143 → 110,139,
128,143 → 124,139,
2,143,130 → 144,139,126,


In [225]:
','.join([str(i) for i in new_restraint_residues])

'144,139,126'

In [258]:
# with open('generated/a-bam-p/disang.rest', 'w') as my_disang:
#     # my_disang.write(lines[0])
#     for line in lines[1:]:
#         strings = line.split()
#         # Grab residues involved in this restraint
#         old_restraint_list = strings[2]
#         old_restraint_residues = [int(i) for i in old_restraint_list.split(',') if i is not '']
        
#         print(line, end="")
        
#         match_string = 'iat= {} {}'
#         line = line.replace()
#         # for atom_index in old_restraint_residues:
#             #print(f'{str(atom_index)} {str(mapping[atom_index - 1])}')
#         #    line = line.replace(',' + str(atom_index), ',' + str(mapping[atom_index - 1]))
#         print(line, end="")

THe problem with find and replace is that it will replace on substrings inside of multidigit numbers, which is really bad.

In [298]:
with open('generated/a-bam-p/disang.rest', 'w') as my_disang:
    my_disang.write(lines[0])

    for line in lines[1:]:
        # Restraint residues...
        old_restraint_list = line.split()[2]
        old_restraint_residues = [int(i) for i in old_restraint_list.split(',') if i is not '']

        # Make a list for the new residues...
        new_restraint_residues = []

        for atom_index in old_restraint_residues:
            new_restraint_residues.append(mapping[atom_index - 1])

        assert len(old_restraint_residues) == len(new_restraint_residues)
        print(f'{old_restraint_list} → ', end="")
        [print(f'{i},', end='') for i in new_restraint_residues]
        print('')
        
        new_restraint_string = ','.join([str(i) for i in new_restraint_residues])
        new_line = line[0:10] + '{0: <18}'.format(new_restraint_string) + line[27:]
        # print(new_line, end="")
        my_disang.write(new_line)

1,13, → 143,9,
2,1,13, → 144,143,9,
3,2,1,13, → 145,144,143,9,
1,13,46, → 143,9,42,
2,1,13,46, → 144,143,9,42,
1,13,46,104, → 143,9,42,100,
1,143, → 143,139,
2,1,143, → 144,143,139,
1,143,130, → 143,139,126,
19,4,6,36, → 15,0,2,32,
40,25,27,57, → 36,21,23,53,
61,46,48,78, → 57,42,44,74,
82,67,69,99, → 78,63,65,95,
103,88,90,120, → 99,84,86,116,
124,109,111,15, → 120,105,107,11,
4,6,36,38, → 0,2,32,34,
25,27,57,59, → 21,23,53,55,
46,48,78,80, → 42,44,74,76,
67,69,99,101, → 63,65,95,97,
88,90,120,122, → 84,86,116,118,
109,111,15,17, → 105,107,11,13,
9,143 → 5,139,
23,143 → 19,139,
30,143 → 26,139,
44,143 → 40,139,
51,143 → 47,139,
65,143 → 61,139,
72,143 → 68,139,
86,143 → 82,139,
93,143 → 89,139,
107,143 → 103,139,
114,143 → 110,139,
128,143 → 124,139,
2,143,130 → 144,139,126,


In [273]:
len(new_line[10:20])

10

In [272]:
len(new_restraint_string.ljust(10))

10

In [284]:
from utils import copy_box

In [297]:
copy_box('original/a-bam-p/full.crds', 'generated/a-bam-p/solvated_smirnoff.inpcrd')

In [542]:
from utils import rewrite_amber_input_file

In [543]:
rewrite_amber_input_file('generated/a-bam-p/mini.in', 'generated/a-bam-p/mini.in', mapping)

["':1-3", '|', ':10@C4', '|', ":10@N1',"] → ':143-145 | :6@C4 | :6@N1,'


In [365]:
def split(delimiters, string, maxsplit=0):
    import re
    regexPattern = '|'.join(map(re.escape, delimiters))
    return re.split(regexPattern, string, maxsplit)

In [450]:
rewrite_amber_input_file('generated/a-bam-p/mini.in', 'tmp.in', mapping)

["':1-3", '|', ':10@C4', '|', ":10@N1',"]
{0: '1-3', 1: '|', 2: '10@C4', 3: '|', 4: '10@N1'}
Working on...1
Working on...3
[143, 145]
|
Working on...10@C4
[6]
|
Working on...10@N1
[6]


In [546]:
mapping[1]

144

## THe mapping is wrong!Arrggghhhhhhhhhhhhhhhhhhhhhhhh

In [540]:
def rewrite_amber_input_file(input_file, output_file, mapping):
    # First, read the existing file...
    with open(input_file, 'r') as file:
        lines = []
        for line in file:
            lines.append(line)
    
    # Next, rewrite using the fact that Niel has a fixed width for the atom index section...
    with open(output_file, 'w') as file:
        for line_number, line in enumerate(lines):
            
            if 'restraintmask' in line:
                restraint_mask = line.split()[2:]
                restraint_mask_line = line_number
            else:
                pass
    
        new_masks = []
        for mask in restraint_mask:
            if ':' in mask:
                if "'" in mask:
                    mask = mask.replace("'", "")
                if "," in mask:
                    mask = mask.replace(",", "")
                # Now we have masks joined by '|'                
                # Split into residues...
                if '-' in mask[1:]:
                    residues = split('-', mask[1:])
                    new_residues = []
                    for residue in residues:
                        new_residue = mapping[int(residue) - 1]
                        new_residues.append(str(new_residue))
                else:
                    residues = split('@', mask[1:])
                    new_residues = []
                    for residue in residues[:1]:
                        new_residue = mapping[int(residue) - 1]
                        new_residues.append(str(new_residue))
                if len(new_residues) > 1:
                    new_mask = ':' + '-'.join(new_residues)
                else:
                    new_mask = ':' + new_residues[0] + '@' + residues[1:][0]
                new_masks.append(new_mask)
            
        new_restraint_mask = '\'' + ' | '.join(new_masks) + ',' + '\''
        print(f'{restraint_mask} → {new_restraint_mask}')
        
        lines[restraint_mask_line] = '  restraintmask = ' + new_restraint_mask + '\n'
        for line in lines:
            file.write(line)


In [541]:
rewrite_amber_input_file('generated/a-bam-p/mini.in', 'tmp.in', mapping)

["':1-3", '|', ':10@C4', '|', ":10@N1',"] → ':143-145 | :6@C4 | :6@N1,'


In [469]:
mapping(int(1) - 1)

TypeError: 'dict' object is not callable

In [548]:
mapping[0]

143

It is atom indexing, not residue!

In [554]:
original = pmd.load_file('original/a-bam-p/full.topo', 'original/a-bam-p/full.crds')
# original.save('tmp.pdb')
# original = pmd.load_file('tmp.pdb')
# original_OEMol = openmmTop_to_oemol(original, original.positions)
# generated_OEMol = openmmTop_to_oemol(generated, generated.positions)

In [641]:
def residue_mapping(reference, target, filter_residue=False):
    """
    Maps between a reference molecule and target molecule using maximum common substructure. For more information, see the example here: https://github.com/openforcefield/openforcefield/blob/6229a51ad77fd5cf20299e53bc9784811cb9443a/openforcefield/typing/engines/smirnoff/forcefield.py#L350
    
    Parameters:
    ----------
    reference : openeye.oechem.OEMol
        Reference molecule for mapping
    target : openeye.oechem.OEMol
        Target molecule for mapping
    Returns
    -------
    dictionary
        The mapping between atom numbers in each molecule
    """

    reference_topology = create_topology(reference)
    target_topology = create_topology(target)

    reference_graph = create_graph(reference_topology)
    target_graph = create_graph(target_topology)

    reference_to_target_mapping = dict()
    graph_matcher = isomorphism.GraphMatcher(reference_graph, target_graph)
    if graph_matcher.is_isomorphic():
        print('Determining mapping...')
        print('Reference → Target')
        for (reference_atom, target_atom) in graph_matcher.mapping.items():

            reference_name = reference.GetAtom(
                OEHasAtomIdx(reference_atom)).GetName()
            reference_type = reference.GetAtom(
                OEHasAtomIdx(reference_atom)).GetType()
                
            reference = reference.GetAtom(OEHasAtomIdx(reference_atom))
            reference_residue = OEAtomGetResidue(reference)
            reference_resname = reference_residue.GetName()
            reference_resnum = reference_residue.GetResidueNumber()

            target_name = target.GetAtom(OEHasAtomIdx(target_atom)).GetName()
            target = target.GetAtom(OEHasAtomIdx(target_atom))
            target_residue = OEAtomGetResidue(target)
            target_resname = target_residue.GetName()
            target_resnum = target_residue.GetResidueNumber()
            
            reference_to_target_mapping[reference_resnum] = target_resnum



            print(
                f'{reference_resname:4} {reference_resnum:3d} → '
                f'{target_resname:4} {target_resnum:3d}')
    else:
        print('Graph is not isomorphic.')

    return reference_to_target_mapping


In [642]:
from utils import atom_mapping, residue_mapping
original = pmd.load_file('original/a-bam-p/full.topo', 'original/a-bam-p/full.crds')
new = pmd.load_file('generated/a-bam-p/solvated_smirnoff.prmtop', 'generated/a-bam-p/solvated_smirnoff.inpcrd')

# original.save('tmp.pdb')
# new.save('tmp2.pdb')

from utils import load_pdb
original = load_pdb('tmp.mol2')
new = load_pdb('tmp2.mol2')

# atom_mapping(original, new)
residue_mapping(original, new)

NotImplementedError: Wrong number or type of arguments for overloaded function 'new_OEMol'.
  Possible C/C++ prototypes are:
    OEMolWrapper::OEMolWrapper(unsigned int)
    OEMolWrapper::OEMolWrapper()
    OEMolWrapper::OEMolWrapper(OEMolWrapper const &)
    OEMolWrapper::OEMolWrapper(OEMolWrapper const &,unsigned int)
    OEMolWrapper::OEMolWrapper(OESystem::OEBase const &,unsigned int)
    OEMolWrapper::OEMolWrapper(OESystem::OEBase const &)
    OEMolWrapper::OEMolWrapper(OEChem::OEMolBase const &,unsigned int)
    OEMolWrapper::OEMolWrapper(OEChem::OEMolBase const &)
    OEMolWrapper::OEMolWrapper(OEChem::OEQMolBase const &,unsigned int)
    OEMolWrapper::OEMolWrapper(OEChem::OEQMolBase const &)
    OEMolWrapper::OEMolWrapper(OEChem::OEMCMolBase const &)
    OEMolWrapper::OEMolWrapper(OEChem::OEMCMolBase const &,unsigned int)


In [582]:
original.GetAtom(OEHasAtomIdx(0)).GetName()

'C1'

In [591]:
reference = original.GetAtom(OEHasAtomIdx(40))
reference_residue = OEAtomGetResidue(reference)
reference_resname = reference_residue.GetName()
reference_resnum = reference_residue.GetResidueNumber()

In [592]:
print(reference_resname, reference_resnum)

UNL 1


In [595]:
original.GetMaxAtomIdx()

126

In [597]:
new.GetMaxAtomIdx()

126

In [615]:
original = load_pdb('tmp.mol2')
original

[<openeye.oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7f3841ecaa50> >]

In [618]:
original[1].GetMaxAtomIdx()

IndexError: list index out of range

In [648]:
from openeye.oechem import *

def load_pdb(file):

    ifs = oemolistream(file)
    #flavor = oechem.OEIFlavor_Generic_Default | oechem.OEIFlavor_PDB_Default | oechem. OEIFlavor_PDB_TER | oechem.OEIFlavor_PDB_ALL
    #ifs.SetFlavor(OEFormat_PDB, flavor)

    for mol in ifs.GetOEGraphMols():
#         print(mol.GetMaxAtomIdx())
#         reference = mol.GetAtom(OEHasAtomIdx(0))
#         reference_residue = OEAtomGetResidue(reference)
#         reference_resname = reference_residue.GetName()
#         reference_resnum = reference_residue.GetResidueNumber()
#         print(reference_resname)
#         print(reference_resnum)
        OETriposAtomNames(mol)
        return mol

In [649]:
original = load_pdb('tmp.pdb')
new = load_pdb('tmp2.pdb')

In [653]:
from utils import residue_mapping
residue_mapping(original, new)

Determining mapping...
Reference → Target
Failing reference atom = 0
MGO    4 → MGO    1
Failing reference atom = 1
MGO    4 → MGO    1
Failing reference atom = 2
MGO    4 → MGO    1
Failing reference atom = 3
MGO    4 → MGO    1
Failing reference atom = 4
MGO    4 → MGO    1
Failing reference atom = 5
MGO    4 → MGO    1
Failing reference atom = 6
MGO    4 → MGO    1
Failing reference atom = 7
MGO    4 → MGO    1
Failing reference atom = 8
MGO    4 → MGO    1
Failing reference atom = 9
MGO    4 → MGO    1
Failing reference atom = 10
MGO    4 → MGO    1
Failing reference atom = 11
MGO    4 → MGO    1
Failing reference atom = 12
MGO    4 → MGO    1
Failing reference atom = 13
MGO    4 → MGO    1
Failing reference atom = 14
MGO    4 → MGO    1
Failing reference atom = 15
MGO    4 → MGO    1
Failing reference atom = 16
MGO    4 → MGO    1
Failing reference atom = 17
MGO    4 → MGO    1
Failing reference atom = 18
MGO    4 → MGO    1
Failing reference atom = 19
MGO    4 → MGO    1
Failing 

WAT  325 → WAT  325
Failing reference atom = 1060
WAT  325 → WAT  325
Failing reference atom = 1061
WAT  325 → WAT  325
Failing reference atom = 1062
WAT  326 → WAT  326
Failing reference atom = 1063
WAT  326 → WAT  326
Failing reference atom = 1064
WAT  326 → WAT  326
Failing reference atom = 1065
WAT  327 → WAT  327
Failing reference atom = 1066
WAT  327 → WAT  327
Failing reference atom = 1067
WAT  327 → WAT  327
Failing reference atom = 1068
WAT  328 → WAT  328
Failing reference atom = 1069
WAT  328 → WAT  328
Failing reference atom = 1070
WAT  328 → WAT  328
Failing reference atom = 1071
WAT  329 → WAT  329
Failing reference atom = 1072
WAT  329 → WAT  329
Failing reference atom = 1073
WAT  329 → WAT  329
Failing reference atom = 1074
WAT  330 → WAT  330
Failing reference atom = 1075
WAT  330 → WAT  330
Failing reference atom = 1076
WAT  330 → WAT  330
Failing reference atom = 1077
WAT  331 → WAT  331
Failing reference atom = 1078
WAT  331 → WAT  331
Failing reference atom = 1079


WAT  632 → WAT  632
Failing reference atom = 1983
WAT  633 → WAT  633
Failing reference atom = 1984
WAT  633 → WAT  633
Failing reference atom = 1985
WAT  633 → WAT  633
Failing reference atom = 1986
WAT  634 → WAT  634
Failing reference atom = 1987
WAT  634 → WAT  634
Failing reference atom = 1988
WAT  634 → WAT  634
Failing reference atom = 1989
WAT  635 → WAT  635
Failing reference atom = 1990
WAT  635 → WAT  635
Failing reference atom = 1991
WAT  635 → WAT  635
Failing reference atom = 1992
WAT  636 → WAT  636
Failing reference atom = 1993
WAT  636 → WAT  636
Failing reference atom = 1994
WAT  636 → WAT  636
Failing reference atom = 1995
WAT  637 → WAT  637
Failing reference atom = 1996
WAT  637 → WAT  637
Failing reference atom = 1997
WAT  637 → WAT  637
Failing reference atom = 1998
WAT  638 → WAT  638
Failing reference atom = 1999
WAT  638 → WAT  638
Failing reference atom = 2000
WAT  638 → WAT  638
Failing reference atom = 2001
WAT  639 → WAT  639
Failing reference atom = 2002


WAT  883 → WAT  883
Failing reference atom = 2735
WAT  883 → WAT  883
Failing reference atom = 2736
WAT  884 → WAT  884
Failing reference atom = 2737
WAT  884 → WAT  884
Failing reference atom = 2738
WAT  884 → WAT  884
Failing reference atom = 2739
WAT  885 → WAT  885
Failing reference atom = 2740
WAT  885 → WAT  885
Failing reference atom = 2741
WAT  885 → WAT  885
Failing reference atom = 2742
WAT  886 → WAT  886
Failing reference atom = 2743
WAT  886 → WAT  886
Failing reference atom = 2744
WAT  886 → WAT  886
Failing reference atom = 2745
WAT  887 → WAT  887
Failing reference atom = 2746
WAT  887 → WAT  887
Failing reference atom = 2747
WAT  887 → WAT  887
Failing reference atom = 2748
WAT  888 → WAT  888
Failing reference atom = 2749
WAT  888 → WAT  888
Failing reference atom = 2750
WAT  888 → WAT  888
Failing reference atom = 2751
WAT  889 → WAT  889
Failing reference atom = 2752
WAT  889 → WAT  889
Failing reference atom = 2753
WAT  889 → WAT  889
Failing reference atom = 2754


WAT  1152 → WAT  1152
Failing reference atom = 3542
WAT  1152 → WAT  1152
Failing reference atom = 3543
WAT  1153 → WAT  1153
Failing reference atom = 3544
WAT  1153 → WAT  1153
Failing reference atom = 3545
WAT  1153 → WAT  1153
Failing reference atom = 3546
WAT  1154 → WAT  1154
Failing reference atom = 3547
WAT  1154 → WAT  1154
Failing reference atom = 3548
WAT  1154 → WAT  1154
Failing reference atom = 3549
WAT  1155 → WAT  1155
Failing reference atom = 3550
WAT  1155 → WAT  1155
Failing reference atom = 3551
WAT  1155 → WAT  1155
Failing reference atom = 3552
WAT  1156 → WAT  1156
Failing reference atom = 3553
WAT  1156 → WAT  1156
Failing reference atom = 3554
WAT  1156 → WAT  1156
Failing reference atom = 3555
WAT  1157 → WAT  1157
Failing reference atom = 3556
WAT  1157 → WAT  1157
Failing reference atom = 3557
WAT  1157 → WAT  1157
Failing reference atom = 3558
WAT  1158 → WAT  1158
Failing reference atom = 3559
WAT  1158 → WAT  1158
Failing reference atom = 3560
WAT  1158 → 

WAT  1419 → WAT  1419
Failing reference atom = 4343
WAT  1419 → WAT  1419
Failing reference atom = 4344
WAT  1420 → WAT  1420
Failing reference atom = 4345
WAT  1420 → WAT  1420
Failing reference atom = 4346
WAT  1420 → WAT  1420
Failing reference atom = 4347
WAT  1421 → WAT  1421
Failing reference atom = 4348
WAT  1421 → WAT  1421
Failing reference atom = 4349
WAT  1421 → WAT  1421
Failing reference atom = 4350
WAT  1422 → WAT  1422
Failing reference atom = 4351
WAT  1422 → WAT  1422
Failing reference atom = 4352
WAT  1422 → WAT  1422
Failing reference atom = 4353
WAT  1423 → WAT  1423
Failing reference atom = 4354
WAT  1423 → WAT  1423
Failing reference atom = 4355
WAT  1423 → WAT  1423
Failing reference atom = 4356
WAT  1424 → WAT  1424
Failing reference atom = 4357
WAT  1424 → WAT  1424
Failing reference atom = 4358
WAT  1424 → WAT  1424
Failing reference atom = 4359
WAT  1425 → WAT  1425
Failing reference atom = 4360
WAT  1425 → WAT  1425
Failing reference atom = 4361
WAT  1425 → 

WAT  1678 → WAT  1678
Failing reference atom = 5120
WAT  1678 → WAT  1678
Failing reference atom = 5121
WAT  1679 → WAT  1679
Failing reference atom = 5122
WAT  1679 → WAT  1679
Failing reference atom = 5123
WAT  1679 → WAT  1679
Failing reference atom = 5124
WAT  1680 → WAT  1680
Failing reference atom = 5125
WAT  1680 → WAT  1680
Failing reference atom = 5126
WAT  1680 → WAT  1680
Failing reference atom = 5127
WAT  1681 → WAT  1681
Failing reference atom = 5128
WAT  1681 → WAT  1681
Failing reference atom = 5129
WAT  1681 → WAT  1681
Failing reference atom = 5130
WAT  1682 → WAT  1682
Failing reference atom = 5131
WAT  1682 → WAT  1682
Failing reference atom = 5132
WAT  1682 → WAT  1682
Failing reference atom = 5133
WAT  1683 → WAT  1683
Failing reference atom = 5134
WAT  1683 → WAT  1683
Failing reference atom = 5135
WAT  1683 → WAT  1683
Failing reference atom = 5136
WAT  1684 → WAT  1684
Failing reference atom = 5137
WAT  1684 → WAT  1684
Failing reference atom = 5138
WAT  1684 → 

WAT  1921 → WAT  1921
Failing reference atom = 5848
WAT  1921 → WAT  1921
Failing reference atom = 5849
WAT  1921 → WAT  1921
Failing reference atom = 5850
WAT  1922 → WAT  1922
Failing reference atom = 5851
WAT  1922 → WAT  1922
Failing reference atom = 5852
WAT  1922 → WAT  1922
Failing reference atom = 5853
WAT  1923 → WAT  1923
Failing reference atom = 5854
WAT  1923 → WAT  1923
Failing reference atom = 5855
WAT  1923 → WAT  1923
Failing reference atom = 5856
WAT  1924 → WAT  1924
Failing reference atom = 5857
WAT  1924 → WAT  1924
Failing reference atom = 5858
WAT  1924 → WAT  1924
Failing reference atom = 5859
WAT  1925 → WAT  1925
Failing reference atom = 5860
WAT  1925 → WAT  1925
Failing reference atom = 5861
WAT  1925 → WAT  1925
Failing reference atom = 5862
WAT  1926 → WAT  1926
Failing reference atom = 5863
WAT  1926 → WAT  1926
Failing reference atom = 5864
WAT  1926 → WAT  1926
Failing reference atom = 5865
WAT  1927 → WAT  1927
Failing reference atom = 5866
WAT  1927 → 

{4: 1,
 5: 2,
 6: 3,
 7: 4,
 8: 5,
 9: 6,
 10: 7,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51,
 52: 52,
 53: 53,
 54: 54,
 55: 55,
 56: 56,
 57: 57,
 58: 58,
 59: 59,
 60: 60,
 61: 61,
 62: 62,
 63: 63,
 64: 64,
 65: 65,
 66: 66,
 67: 67,
 68: 68,
 69: 69,
 70: 70,
 71: 71,
 72: 72,
 73: 73,
 74: 74,
 75: 75,
 76: 76,
 77: 77,
 78: 78,
 79: 79,
 80: 80,
 81: 81,
 82: 82,
 83: 83,
 84: 84,
 85: 85,
 86: 86,
 87: 87,
 88: 88,
 89: 89,
 90: 90,
 91: 91,
 92: 92,
 93: 93,
 94: 94,
 95: 95,
 96: 96,
 97: 97,
 98: 98,
 99: 99,
 100: 100,
 101: 101,
 102: 102,
 103: 103,
 104: 104,
 105: 105,
 106: 106,
 107: 107,
 108: 108,
 109: 109,
 110: 110,
 111: 111,
 112: 112,
 113: 

In [654]:
mapping

{3: 0,
 4: 1,
 5: 2,
 6: 3,
 7: 4,
 8: 5,
 9: 6,
 10: 7,
 11: 8,
 12: 9,
 13: 10,
 14: 11,
 15: 12,
 16: 13,
 17: 14,
 18: 15,
 19: 16,
 20: 17,
 21: 18,
 22: 19,
 23: 20,
 35: 32,
 31: 28,
 27: 24,
 24: 21,
 25: 22,
 26: 23,
 28: 25,
 29: 26,
 30: 27,
 32: 29,
 33: 30,
 34: 31,
 36: 33,
 37: 34,
 38: 35,
 39: 36,
 40: 37,
 41: 38,
 42: 39,
 43: 40,
 44: 41,
 56: 53,
 52: 49,
 48: 45,
 45: 42,
 46: 43,
 47: 44,
 49: 46,
 50: 47,
 51: 48,
 53: 50,
 54: 51,
 55: 52,
 57: 54,
 58: 55,
 59: 56,
 60: 57,
 61: 58,
 62: 59,
 63: 60,
 64: 61,
 65: 62,
 77: 74,
 73: 70,
 69: 66,
 66: 63,
 67: 64,
 68: 65,
 70: 67,
 71: 68,
 72: 69,
 74: 71,
 75: 72,
 76: 73,
 78: 75,
 79: 76,
 80: 77,
 81: 78,
 82: 79,
 83: 80,
 84: 81,
 85: 82,
 86: 83,
 98: 95,
 94: 91,
 90: 87,
 87: 84,
 88: 85,
 89: 86,
 91: 88,
 92: 89,
 93: 90,
 95: 92,
 96: 93,
 97: 94,
 99: 96,
 100: 97,
 101: 98,
 102: 99,
 103: 100,
 104: 101,
 105: 102,
 106: 103,
 107: 104,
 110: 107,
 108: 105,
 109: 106,
 111: 108,
 112: 109,
 113

In [ ]:
for (key, value) in mapping.items():
        reference = mol.GetAtom(OEHasAtomIdx(0))
    reference_residue = OEAtomGetResidue(reference)
    reference_resname = reference_residue.GetName()
    reference_resnum = reference_residue.GetResidueNumber()
